### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Anxiety_disorder/GSE68526'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# 1. Gene Expression Data Availability
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Define lookup keys for Anxiety_disorder, age, and gender
import re

# Anxiety_disorder: Based on Beck Anxiety Inventory, represented as "anxiety"
trait_row = 13 if len(set(re.sub(r"anxiety: ", "", val) for val in sample_characteristics[13])) > 1 else None

# Age variable: Check unique values availability
age_row = 0 if len(set(re.sub(r"age \(yrs\): ", "", val) for val in sample_characteristics[0])) > 1 else None

# Gender variable: Check unique values availability
gender_row = 1 if len(set(re.sub(r"female: ", "", val) for val in sample_characteristics[1])) > 1 else None

# Functions for data type conversion
def convert_trait(value):
    value = value.split(":")[1].strip()
    try:
        return float(value) if value != "missing" else None
    except ValueError:
        return None

def convert_age(value):
    value = value.split(":")[1].strip()
    try:
        return int(value) if value != "missing" else None
    except ValueError:
        return None

def convert_gender(value):
    value = value.split(":")[1].strip()
    if value == "0":
        return 0
    elif value == "1":
        return 1
    else:
        return None

# 3. Save Metadata
save_cohort_info('GSE68526', './preprocessed/Anxiety_disorder/cohort_info.json', is_gene_available, trait_row is not None)

# 4. Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Anxiety_disorder', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Anxiety_disorder/trait_data/GSE68526.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
